# Purpose: 
1. Process Google Form Responses from pool play
2. output record info (wins/losses, win ratio/loss ratio, points) for display
3. after pool play is over, output record info for seeding next pool play

### To Do:
1. Modify for day 2
2. input from day 2 round robin
3. output to playoff brackets

### Setup output, get team names

In [104]:
from ReadWriteGDocs import write_gdoc, read_gdoc_range, write_gdoc_cell

num_teams = 49 ### 49 for men, 45 for women ###
master_sheet = "Men's Master" ### Change for men vs women ###

num_gold = 20
num_silver = 15

import pandas as pd
import numpy as np

input_ws = "NY Mini 2019 Score Input - Men's (Responses)" ### Change for men vs women ###
output_ws = ["NY Mini 2019 - 1st Round", "NY Mini 2019 - 2nd Round"]

c=read_gdoc_range(input_ws, "Form Responses 1", 1,2,1,12)
c=c[0]

# get list of team names from Form Values
results = pd.DataFrame(read_gdoc_range("Form Values Add-on - DO NOT DELETE", "Sheet1", 2, 4 ### 4 for men, 5 for women ###
                                       , 1+num_teams, 4 ### 4 for men, 5 for women ###
                                      ), columns=["Team Name"])

results["Court Number"] = ""
results["Wins"] = 0
results["Points"] = 0
results["Pool Ranking"] = 0

# get court numbers from master day 1
courts = pd.DataFrame(read_gdoc_range(output_ws[0], master_sheet, 3, 2, 9, 10))

# iterate each row
for i in range(1,courts.shape[0]):
    for j in range(0, courts.shape[1]):
        team_name = courts.at[i, j]
        
        if (team_name != "X"):
            team_row = results.loc[results["Team Name"] == team_name]
            results.at[team_row.index, "Court Number"] = courts.at[0, j]
        
# start processing from row 2
# keep track of rows processed using processed

processed = 0

### Import new Google Form responses

In [105]:
responses_in = pd.DataFrame(read_gdoc_range(input_ws, "Form Responses 1", processed + 2,2, 200, 12), 
                    columns=c)

responses_in.replace('', np.nan, inplace=True)
responses_in.dropna(inplace=True, how='all')

from math import isnan

print ("Need to process " + str(len(responses_in)) + " games")

Need to process 87 games


### Process each Google Form response - update wins and points for teams

In [106]:
responses_trimmed = responses_in.drop(c[2:5], axis=1)
## Convert to numeric
for i in responses_trimmed.columns:
    responses_trimmed[i] = pd.to_numeric(responses_trimmed[i], downcast='integer', errors='ignore')
    
# iterate through each row of responses_trimmed and see if team name is found

for index,row in responses_trimmed.iterrows():
    team_1_name = row["Team 1 Name"]
    team_2_name = row["Team 2 Name"]
    win1 = 0
    win2 = 0
    points1 = 0
    points2 = 0
    
    # process first set
    score_diff = row["Team 1 Score"] - row["Team 2 Score"]
    points1 += score_diff
    points2 -= score_diff
        
    if (score_diff > 0):
        win1 += 1
    elif (score_diff < 0):
        win2 += 1
    ## if score_diff == 0 then do nothing
        
    # process second set
    score_diff = row["Team 1 Score 2"] - row["Team 2 Score 2"]
    points1 += score_diff
    points2 -= score_diff
    
    if (score_diff > 0): # team 1 won
        win1 += 1
    elif (score_diff < 0): # team 2 won
        win2 += 1
    ## if score_diff == 0 then do nothing
                    
    team_1_row = results.loc[results["Team Name"] == team_1_name]
    results.at[team_1_row.index, "Wins"] = team_1_row["Wins"] + win1
    results.at[team_1_row.index, "Points"] = team_1_row["Points"] + points1
    
    team_2_row = results.loc[results["Team Name"] == team_2_name]
    results.at[team_2_row.index, "Wins"] = team_2_row["Wins"] + win2
    results.at[team_2_row.index, "Points"] = team_2_row["Points"] + points2
    
    if (team_1_name == "NY Strangers Black"):
        print (str(team_1_row["Points"]))
        print (str(win1) + ", " + str(points1) + "\n")
    
# add to results: wins and points

    processed = processed + 1
        
print("Number of records processed: " + str(len(responses_trimmed)))

print("Total Processed: " + str(processed))

27    0
Name: Points, dtype: int64
2, 16

27    16
Name: Points, dtype: int64
2, 10

27    26
Name: Points, dtype: int64
2, 28

27    54
Name: Points, dtype: int64
2, 15

Number of records processed: 87
Total Processed: 87


In [107]:
#results.sort_values(by=["Wins", "Points"], ascending=[0,0])
results.sort_values(by=["Court Number", "Wins", "Points"])

,Team Name,Court Number,Wins,Points,Pool Ranking
30,NY Strangers White,CT01,0,-55,0
1,Boston Hurricanes Blue,CT01,0,-51,0
10,DC CYC B,CT01,3,-19,0
32,NY Vikings SPA,CT01,3,2,0
38,Philly Super CIA,CT01,6,37,0
39,Toronto Connex A,CT01,8,86,0
6,Boston Knights X,CT02,0,-69,0
24,NY Impact,CT02,2,-49,0
25,NY Panda Glass,CT02,4,11,0
15,DC Yee Fung Toy Thunder A,CT02,6,28,0


### Sort results by court number, team name

### Update display for standings

In [108]:
# sort by court number and team name

disp_results = results.sort_values(by=["Court Number", "Team Name"]).reset_index(drop=True)

#update display output - court numbers and team names - one time only
# for i in range(0, len(disp_results)):
#     write_gdoc_cell(disp_results.at[i, "Court Number"], output_ws[0], "ScoreDisp", 5+i, 9)  ## 2 for women, 9 for women
#     write_gdoc_cell(disp_results.at[i, "Team Name"], output_ws[0], "ScoreDisp", 5+i, 10) ## 3 for men, 10 for women
    
# # update display output - wins and points - keep updating
for i in range(0, len(disp_results)):
    write_gdoc_cell(int(disp_results.at[i, "Wins"]), output_ws[0], "ScoreDisp", 5+i, 4) ## 4 for men, 11 for women
    write_gdoc_cell(int(disp_results.at[i, "Points"]), output_ws[0], "ScoreDisp", 5+i, 5) ## 5 for men, 12 for women

APIError: {
  "error": {
    "code": 429,
    "message": "Quota exceeded for quota group 'ReadGroup' and limit 'USER-100s' of service 'sheets.googleapis.com' for consumer 'project_number:371878011270'.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Google developer console API key",
            "url": "https://console.developers.google.com/project/371878011270/apiui/credential"
          }
        ]
      }
    ]
  }
}


### Sort results by court number, wins, points

### Write team names to Gold, Silver, Bronze brackets

# Need to adjust - top 2 teams from each pool goes into Gold, rest are by overall ranking
## For 2nd and 3rd placed teams with the same #games won, need to check head to head

In [31]:
# sort by court number, wins, points for seeding
results = results.sort_values(by=["Court Number", "Wins", "Points"], ascending=[1,0,0]).reset_index(drop=True)

## top 20 - gold - first two teams of each pool and then the top 2 from the rest
## next 15 - silver - next 15
## bottom 14 (men's) or 10 (women's) - bronze

# assign pool rankings

for court in results["Court Number"].unique():
    # retrieve pool
    pool = results[results["Court Number"] == court]
    # sort by wins and points
    pool = pool.sort_values(by=["Wins", "Points"], ascending=[0,0]).reset_index(drop=True)
    
    # assign rankings
    for i in range(0, len(pool)):
        results.loc[results["Team Name"] == pool.loc[i, "Team Name"], "Pool Ranking"] = i+1

# do not use chained assignments - [] then []
# results["Pool Ranking"][results["Team Name"] == team["Team Name"]] = 4
# use this instead
# results.loc[results["Team Name"] == team["Team Name"], "Pool Ranking"] = 6

## sort by pool ranking, games, points, grab the first 20

results = results.sort_values(by=["Pool Ranking", "Wins", "Points"], ascending=[1,0,0]).reset_index(drop=True)

# get pools
gold_pool = results.loc[0:num_gold-1]
silver_pool = results.loc[num_gold:num_gold+num_silver-1].reset_index(drop=True)
bronze_pool = results.loc[num_gold+num_silver:].reset_index(drop=True)

## snake teams into second day pool play

## gold pools - top 20 - same for men and women

for i in range(0, 5):
    if (i % 2 == 0):
        write_gdoc_cell(gold_pool.at[i*4, "Team Name"], output_ws[1], master_sheet, 5+i, 2)
        write_gdoc_cell(gold_pool.at[i*4+1, "Team Name"], output_ws[1], master_sheet, 5+i, 2+1)
        write_gdoc_cell(gold_pool.at[i*4+2, "Team Name"], output_ws[1], master_sheet, 5+i, 2+2)
        write_gdoc_cell(gold_pool.at[i*4+3, "Team Name"], output_ws[1], master_sheet, 5+i, 2+3)
    else:
        write_gdoc_cell(gold_pool.at[i*4+3, "Team Name"], output_ws[1], master_sheet, 5+i, 2)
        write_gdoc_cell(gold_pool.at[i*4+2, "Team Name"], output_ws[1], master_sheet, 5+i, 2+1)
        write_gdoc_cell(gold_pool.at[i*4+1, "Team Name"], output_ws[1], master_sheet, 5+i, 2+2)
        write_gdoc_cell(gold_pool.at[i*4+0, "Team Name"], output_ws[1], master_sheet, 5+i, 2+3)
        
## silver pools - next 15 - same for men and women

for i in range(0, 5):
    if (i % 2 == 0):
        write_gdoc_cell(silver_pool.at[i*3, "Team Name"], output_ws[1], master_sheet, 5+i, 8)
        write_gdoc_cell(silver_pool.at[i*3+1, "Team Name"], output_ws[1], master_sheet, 5+i, 8+1)
        write_gdoc_cell(silver_pool.at[i*3+2, "Team Name"], output_ws[1], master_sheet, 5+i, 8+2)
    else:
        write_gdoc_cell(silver_pool.at[i*3+2, "Team Name"], output_ws[1], master_sheet, 5+i, 8+0)
        write_gdoc_cell(silver_pool.at[i*3+1, "Team Name"], output_ws[1], master_sheet, 5+i, 8+1)
        write_gdoc_cell(silver_pool.at[i*3+0, "Team Name"], output_ws[1], master_sheet, 5+i, 8+2)
        
## men's bronze pools - no snaking

for i in range (0, 7):
    write_gdoc_cell(bronze_pool.at[i, "Team Name"], output_ws[1], master_sheet, 5+i, 13)
    
for i in range (0, 7):
    write_gdoc_cell(bronze_pool.at[7+i, "Team Name"], output_ws[1], master_sheet, 5+i, 14)
    
## women's bronze pools - 10 teams snaking

# for i in range (0, 5):
#     if (i % 2 == 0):
#         write_gdoc_cell(bronze_pool.at[20+15+i*2, "Team Name"], output_ws[1], master_sheet, 5+i, 13)
#         write_gdoc_cell(bronze_pool.at[20+15+i*2+1, "Team Name"], output_ws[1], master_sheet, 5+i, 13+1)
#     else:
#         write_gdoc_cell(bronze_pool.at[20+15+i*2+1, "Team Name"], output_ws[1], master_sheet, 5+i, 13+1)
#         write_gdoc_cell(bronze_pool.at[20+15+i*2, "Team Name"], output_ws[1], master_sheet, 5+i, 13)